In [2]:
!git clone https://github.com/xinntao/ESRGAN


Cloning into 'ESRGAN'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 225 (delta 15), reused 14 (delta 14), pack-reused 205 (from 1)
Receiving objects: 100% (225/225), 24.86 MiB | 4.53 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [10]:
!pip install opencv-python

  Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-macosx_13_0_arm64.whl (37.3 MB)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import os.path as osp
import cv2
import numpy as np
import torch
import sys
sys.path.append("/Users/harinathreddy/Documents/major_project/ISR/ESRGAN")
import os
import RRDBNet_arch as arch

def enhance_image(input_path, output_dir, model_path, device='cpu'):
    device = torch.device(device)
    model = arch.RRDBNet(3, 3, 64, 23, gc=32)
    model.load_state_dict(torch.load(model_path, map_location=device), strict=True)
    model.eval()
    model = model.to(device)
    os.makedirs(output_dir,exist_ok=True)
    print(f'Loaded model from {model_path}')
    print(f'Enhancing image: {input_path}')

    # Read and preprocess image
    img = cv2.imread(input_path, cv2.IMREAD_COLOR)
    if img is None:
        raise FileNotFoundError(f"Image not found at {input_path}")
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float().unsqueeze(0).to(device)

    # Inference
    with torch.no_grad():
        output = model(img).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round().astype(np.uint8)

    # Construct output filename with same extension

    base_name = osp.splitext(osp.basename(input_path))[0]
    ext = osp.splitext(input_path)[1]
    output_path = osp.join(output_dir, f"{base_name}_rlt{ext}")

    # Save result
    cv2.imwrite(output_path, output)
    print(f'Saved enhanced image to: {output_path}')

# Example usage
input_img = '/Users/harinathreddy/Documents/major_project/ISR/ESRGAN/LR/baboon.png'
output_dir = '/Users/harinathreddy/Documents/major_project/ISR/results'
model_path = '/Users/harinathreddy/Documents/major_project/ISR/ESRGAN/models/RRDB_ESRGAN_x4.pth'

enhance_image(input_img, output_dir, model_path, device='cpu')


/var/folders/0c/lxn_z4x57q1dc5xb66ypsdm00000gn/T/ipykernel_74661/383772227.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, 

Loaded model from /Users/harinathreddy/Documents/major_project/ISR/ESRGAN/models/RRDB_ESRGAN_x4.pth
Enhancing image: /Users/harinathreddy/Documents/major_project/ISR/ESRGAN/LR/baboon.png
Saved enhanced image to: /Users/harinathreddy/Documents/major_project/ISR/results/baboon_rlt.png
